In [1]:
import numpy as np
import pandas as pd

import time

import matplotlib.pyplot as plt
import os
import re
import seaborn as sns


In [2]:
from scipy.stats import moment
from scipy.stats import describe

In [3]:
import nltk
nltk.download('brown')

[nltk_data] Downloading package brown to /home/cdsw/nltk_data...
[nltk_data]   Package brown is already up-to-date!


True

In [4]:
from textblob import TextBlob

In [5]:
from scipy.stats import moment
from scipy.stats import describe


In [6]:
def basic_stats_error(datalist):
    
    if len(datalist)>1:
        result_describe = list(describe(datalist)[2:])
        result_percentile = result_describe + list(np.percentile(datalist,[10,25,50,75,90]))
        array = np.array(datalist)
        result_percentile = result_percentile+[len(array),len(array[np.where(array>0)[0]])/len(array)*100,
                                               len(array[np.where(array<0)[0]])/len(array)*100]
    else:
        result_percentile = []

    return result_percentile

In [7]:
def explode(df, lst_cols, fill_value=''):
    # make sure `lst_cols` is a list
    if lst_cols and not isinstance(lst_cols, list):
        lst_cols = [lst_cols]
    # all columns except `lst_cols`
    idx_cols = df.columns.difference(lst_cols)

    # calculate lengths of lists
    lens = df[lst_cols[0]].str.len()

    if (lens > 0).all():
        # ALL lists in cells aren't empty
        return pd.DataFrame({
            col:np.repeat(df[col].values, df[lst_cols[0]].str.len())
            for col in idx_cols
        }).assign(**{col:np.concatenate(df[col].values) for col in lst_cols}) \
          .loc[:, df.columns]
    else:
        # at least one list in cells is empty
        return pd.DataFrame({
            col:np.repeat(df[col].values, df[lst_cols[0]].str.len())
            for col in idx_cols
        }).assign(**{col:np.concatenate(df[col].values) for col in lst_cols}) \
          .append(df.loc[lens==0, idx_cols]).fillna(fill_value) \
          .loc[:, df.columns]

In [8]:
import seaborn as sns

In [9]:
sns.set(style="whitegrid")

In [10]:
from importlib import reload

from nlp_pandas_functions import ngram_analysis
from nlp_pandas_functions import noun_sentences

reload(ngram_analysis)
reload(noun_sentences)
dir(ngram_analysis)

2022-05-17 01:38:41,598 loading file /home/cdsw/.flair/models/sentiment-en-mix-distillbert_4.pt


INFO:absl:Using /tmp/tfhub_modules to cache modules.


module https://tfhub.dev/google/universal-sentence-encoder/4 loaded
module https://tfhub.dev/google/universal-sentence-encoder/4 loaded


['__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__spec__',
 'bigrams_join',
 'count_n_grams',
 'defaultdict',
 'df_get_n_grams_count',
 'df_lambda_ngram',
 'generate_n_grams',
 'get_imp',
 'get_n_grams_count',
 'get_n_grams_probability',
 'kg',
 'ngrams',
 'np',
 'pd',
 'text']

In [11]:
column = 'clean_publicremarks'

In [12]:
#from nlp_pandas_functions import ngram_analysis
from nlp_pandas_functions import noun_sentences

#reload(ngram_analysis)
reload(noun_sentences)
#dir(ngram_analysis)

module https://tfhub.dev/google/universal-sentence-encoder/4 loaded


<module 'nlp_pandas_functions.noun_sentences' from '/home/cdsw/final_model_lightgbm/nlp_pandas_functions/noun_sentences.py'>

In [27]:
import glob
filtered_files = glob.glob("new_avm_df_state_*_noun_sentences_by_listingid_CJ.fea")

In [28]:
filtered_files

['new_avm_df_state_TN_noun_sentences_noun_sentences_by_listingid_CJ.fea',
 'new_avm_df_state_FL_noun_sentences_noun_sentences_by_listingid_CJ.fea',
 'new_avm_df_state_TX_noun_sentences_noun_sentences_by_listingid_CJ.fea',
 'new_avm_df_state_KS_noun_sentences_noun_sentences_by_listingid_CJ.fea',
 'new_avm_df_state_AZ_noun_sentences_noun_sentences_by_listingid_CJ.fea',
 'new_avm_df_state_GA_noun_sentences_noun_sentences_by_listingid_CJ.fea',
 'new_avm_df_state_OK_noun_sentences_noun_sentences_by_listingid_CJ.fea',
 'new_avm_df_state_UT_noun_sentences_noun_sentences_by_listingid_CJ.fea',
 'new_avm_df_state_MS_noun_sentences_noun_sentences_by_listingid_CJ.fea',
 'new_avm_df_state_WA_noun_sentences_noun_sentences_by_listingid_CJ.fea',
 'new_avm_df_state_OH_noun_sentences_noun_sentences_by_listingid_CJ.fea',
 'new_avm_df_state_NC_noun_sentences_noun_sentences_by_listingid_CJ.fea',
 'new_avm_df_state_CO_noun_sentences_noun_sentences_by_listingid_CJ.fea',
 'new_avm_df_state_MO_noun_sentences_n

In [30]:
noun_sentence_df_T_ri_filtered_thu_list_base= pd.read_csv(filtered_files[0])

In [31]:
filtered_files[0].split('_')[4]

'TN'

In [32]:
noun_sentence_df_T_ri_filtered_thu_list_base['state'] = filtered_files[0].split('_')[4]

In [33]:
noun_sentence_df_T_ri_filtered_thu_list_base

,Unnamed: 0,noun_sentence,listingid,state
0,150574,new hot water heater,2257521,TN
1,154150,car carport,2257521,TN
2,175034,nice home,2257521,TN
3,172359,great location,8663618,TN
4,131015,room fireplace,141342205,TN
...,...,...,...,...
177850,117895,master bedroom,286679688,TN
177851,149520,bath home,286679688,TN
177852,53928,stainless appliances,286679688,TN
177853,92241,tile backsplash,286679688,TN


In [34]:
for file in filtered_files[1:]:
    
    print(file)    
    noun_sentence_df_T_ri_filtered_thu_list= pd.read_csv(file)
    print(len(noun_sentence_df_T_ri_filtered_thu_list))
    noun_sentence_df_T_ri_filtered_thu_list['state'] = file.split('_')[4]
    noun_sentence_df_T_ri_filtered_thu_list_base = noun_sentence_df_T_ri_filtered_thu_list_base.append(noun_sentence_df_T_ri_filtered_thu_list)

    

new_avm_df_state_FL_noun_sentences_noun_sentences_by_listingid_CJ.fea
2006827
new_avm_df_state_TX_noun_sentences_noun_sentences_by_listingid_CJ.fea
2260745
new_avm_df_state_KS_noun_sentences_noun_sentences_by_listingid_CJ.fea
4152
new_avm_df_state_AZ_noun_sentences_noun_sentences_by_listingid_CJ.fea
144458
new_avm_df_state_GA_noun_sentences_noun_sentences_by_listingid_CJ.fea
1344608
new_avm_df_state_OK_noun_sentences_noun_sentences_by_listingid_CJ.fea
21946
new_avm_df_state_UT_noun_sentences_noun_sentences_by_listingid_CJ.fea
23824
new_avm_df_state_MS_noun_sentences_noun_sentences_by_listingid_CJ.fea
20314
new_avm_df_state_WA_noun_sentences_noun_sentences_by_listingid_CJ.fea
337489
new_avm_df_state_OH_noun_sentences_noun_sentences_by_listingid_CJ.fea
51832
new_avm_df_state_NC_noun_sentences_noun_sentences_by_listingid_CJ.fea
310594
new_avm_df_state_CO_noun_sentences_noun_sentences_by_listingid_CJ.fea
597870
new_avm_df_state_MO_noun_sentences_noun_sentences_by_listingid_CJ.fea
61337
new

In [35]:
noun_sentence_df_T_ri_filtered_thu_list_base = noun_sentence_df_T_ri_filtered_thu_list_base.drop(columns=['Unnamed: 0'])

In [36]:
file_join = 'ch_noun_sentences_by_listingid_CJ_2.csv'
noun_sentence_df_T_ri_filtered_thu_list_base.to_csv(file_join)

In [46]:
noun_sentence_df_T_ri_filtered_thu = noun_sentence_df_T_ri_filtered.loc[:,['noun_sentence','Listingid_List']]

In [47]:
noun_sentence_df_T_ri_filtered_thu

,noun_sentence,Listingid_List
162046,new bathrooms,"[226771172, 226894344, 226798752, 226705586, 2..."
191942,measurements approximate,"[226789511, 226972662, 227216981, 227220287, 2..."
204925,top bottom,"[226874336, 227335468, 227067931, 227204798, 2..."
207082,quartz countertops,"[227126688, 227163953, 227060322, 227137424, 2..."
225617,new cabinets,"[226894344, 226755849, 226967145, 226807551, 2..."
...,...,...
808440,great potential,"[227171512, 227359007, 228392861, 228820100, 2..."
809053,major interstates,"[227154427, 226931146, 227598976, 229566962, 2..."
812573,great investment property,"[226801678, 226766720, 227620770, 227835706, 2..."
822110,needs tlc,"[10193559, 228298342, 231697221, 233081549, 23..."


In [48]:
noun_sentence_df_T_ri_filtered_thu_list = explode(noun_sentence_df_T_ri_filtered_thu,
                                           lst_cols=['Listingid_List'])

In [49]:
noun_sentence_df_T_ri_filtered_thu_list

,noun_sentence,Listingid_List
0,new bathrooms,226771172
1,new bathrooms,226894344
2,new bathrooms,226798752
3,new bathrooms,226705586
4,new bathrooms,226773117
...,...,...
1128967,right redemption,10173537
1128968,right redemption,83277261
1128969,right redemption,10176800
1128970,right redemption,10303984
